In [1]:
# --- Setup and Imports ---
import pandas as pd
import numpy as np
import os
import joblib
from datetime import datetime, timedelta

# --- Configuration ---
MODELS_PATH = 'models/'
CROPS = ['Jowar', 'Paddy', 'Maize', 'Cotton']
FEATURES = [
    'fertilizer_kg_ha', 'irrigation_m3_ha', 
    'total_precip_mm', 'avg_temp_max_C', 'total_sunshine_h'
]

# Load all trained models
MODELS = {}
for crop in CROPS:
    model_path = os.path.join(MODELS_PATH, f'{crop.lower()}_model.joblib')
    try:
        MODELS[crop] = joblib.load(model_path)
        print(f"Loaded {crop} prediction model successfully.")
    except FileNotFoundError:
        print(f"❌ ERROR: Model file not found for {crop} at {model_path}. Please re-run Step 3.")
        MODELS[crop] = None

# --- 1. The Generative Scenario Function (Synthetic Data Generator) ---

def generate_future_weather_scenario(start_date, end_date, scenario_type='NORMAL'):
    """
    Generates synthetic aggregated weather data for a future growing period.
    This simulates a Generative AI component (e.g., a simple time-series GAN 
    would replace this in a complex system, but we use statistical generation here).
    """
    duration_days = (end_date - start_date).days
    
    # Base statistics (can be improved with climate trend analysis)
    base_precip = 5.0 * duration_days # Average 5mm/day for Colombia
    base_temp_max = 28.0 
    base_sunshine = 7.0 * duration_days # Average 7 hours/day

    # Apply scenario adjustments (Generative AI logic)
    if scenario_type == 'DROUGHT':
        precip_factor = 0.4 # 60% less rain
        temp_max_adjustment = 3.0 # 3C hotter
    elif scenario_type == 'HEAVY_RAIN':
        precip_factor = 1.6 # 60% more rain
        temp_max_adjustment = -1.5 # 1.5C cooler
    else: # NORMAL (or 'Ideal')
        precip_factor = 1.0
        temp_max_adjustment = 0.0
        
    # Generate the simulated feature values
    scenario_data = {
        'total_precip_mm': base_precip * precip_factor + np.random.uniform(-50, 50),
        'avg_temp_max_C': base_temp_max + temp_max_adjustment + np.random.uniform(-1.0, 1.0),
        'total_sunshine_h': base_sunshine * (1 + np.random.uniform(-0.1, 0.1)) # Small variation
    }
    
    # Ensure no negative values
    for key, value in scenario_data.items():
        scenario_data[key] = max(0, value)
        
    return scenario_data

# --- 2. The Digital Twin Prediction Function (What-If Engine) ---

def predict_yield_scenario(crop, planting_date, harvest_date, management_inputs, scenario_type='NORMAL'):
    """
    Runs a 'What-If' simulation on the digital twin.
    """
    if MODELS.get(crop) is None:
        return f"Model for {crop} is not loaded/trained.", None

    # Step 1: Generate the Future Environment (Generative AI part)
    weather_features = generate_future_weather_scenario(
        planting_date, harvest_date, scenario_type
    )

    # Step 2: Assemble the input features for the trained ML model
    
    # Combine user's management decisions (e.g., fertilizer/irrigation)
    # with the generated weather features
    input_data = {
        'fertilizer_kg_ha': management_inputs.get('fertilizer_kg_ha', 150),
        'irrigation_m3_ha': management_inputs.get('irrigation_m3_ha', 500),
        'total_precip_mm': weather_features['total_precip_mm'],
        'avg_temp_max_C': weather_features['avg_temp_max_C'],
        'total_sunshine_h': weather_features['total_sunshine_h']
    }
    
    # Create a DataFrame in the correct feature order (as used in training)
    X_new = pd.DataFrame([input_data], columns=FEATURES)

    # Step 3: Get the Prediction from the Digital Twin (ML Model)
    predicted_yield = MODELS[crop].predict(X_new)[0]
    
    # Ensure yield is realistic (e.g., non-negative)
    predicted_yield = max(100, predicted_yield) 
    
    # Return results
    return predicted_yield, weather_features

# --- 3. Run Sample Scenario Analysis ---
print("\n--- Running Digital Twin 'What-If' Scenarios (Jowar) ---")

# Base Management Plan for Jowar (current year)
jowar_management = {
    'fertilizer_kg_ha': 160,
    'irrigation_m3_ha': 600,
}
# Future growing season start/end (simulated)
planting_date = datetime(2025, 5, 1)
harvest_date = datetime(2025, 9, 15)

# Scenario A: Normal Weather
yield_normal, weather_normal = predict_yield_scenario(
    'Jowar', planting_date, harvest_date, jowar_management, 'NORMAL'
)
print(f"Scenario A (NORMAL): Predicted Yield = {yield_normal:.2f} kg/ha")
print(f"  > Weather Input: Precip={weather_normal['total_precip_mm']:.0f}mm, TempMaxAvg={weather_normal['avg_temp_max_C']:.1f}C")

# Scenario B: Drought Conditions
yield_drought, weather_drought = predict_yield_scenario(
    'Jowar', planting_date, harvest_date, jowar_management, 'DROUGHT'
)
print(f"\nScenario B (DROUGHT): Predicted Yield = {yield_drought:.2f} kg/ha")
print(f"  > Weather Input: Precip={weather_drought['total_precip_mm']:.0f}mm, TempMaxAvg={weather_drought['avg_temp_max_C']:.1f}C")

# Scenario C: Drought + Increased Irrigation (Mitigation Strategy)
jowar_mitigation = {
    'fertilizer_kg_ha': 160,
    'irrigation_m3_ha': 1200, # Double the irrigation
}
yield_mitigate, weather_mitigate = predict_yield_scenario(
    'Jowar', planting_date, harvest_date, jowar_mitigation, 'DROUGHT'
)
print(f"\nScenario C (DROUGHT + MITIGATION): Predicted Yield = {yield_mitigate:.2f} kg/ha")
print(f"  > Management Change: Irrigation increased to 1200 m3/ha")
print(f"  > **Potential Gain from Mitigation**: {yield_mitigate - yield_drought:.2f} kg/ha")

print("-" * 50)
print("NEXT STEP: Develop the Web Application Interface (Streamlit) for real-time interaction.")

Loaded Jowar prediction model successfully.
Loaded Paddy prediction model successfully.
Loaded Maize prediction model successfully.
Loaded Cotton prediction model successfully.

--- Running Digital Twin 'What-If' Scenarios (Jowar) ---


C:\Documents\AI lab\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Documents\AI lab\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Documents\AI lab\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Scenario A (NORMAL): Predicted Yield = 3266.30 kg/ha
  > Weather Input: Precip=659mm, TempMaxAvg=27.9C

Scenario B (DROUGHT): Predicted Yield = 3244.94 kg/ha
  > Weather Input: Precip=258mm, TempMaxAvg=30.8C

Scenario C (DROUGHT + MITIGATION): Predicted Yield = 3248.45 kg/ha
  > Management Change: Irrigation increased to 1200 m3/ha
  > **Potential Gain from Mitigation**: 3.51 kg/ha
--------------------------------------------------
NEXT STEP: Develop the Web Application Interface (Streamlit) for real-time interaction.
